# Project Check-In 6

**Neural Network**

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering

complete_data = pd.read_csv("StudentPerformanceFactors.csv")

In [5]:
# Clean the data by removing rows with empty values
columns_to_drop = ["Extracurricular_Activities", "Peer_Influence", "Physical_Activity", "Gender"]
data = complete_data.drop(columns=columns_to_drop)
data = data.dropna()
data.isna().sum()
# Convert categorical columns to numerical to support clustering using euclidean (as shown in k-means)
# Convert low medium and high to 0, 1, 2
data['Parental_Involvement'] = data['Parental_Involvement'].map({'Low': 1, 'Medium': 2, 'High': 3})
data['Access_to_Resources'] = data['Access_to_Resources'].map({'Low': 1, 'Medium': 2, 'High': 3})
data['Motivation_Level'] = data['Motivation_Level'].map({'Low': 1, 'Medium': 2, 'High': 3})
data['Family_Income'] = data['Family_Income'].map({'Low': 1, 'Medium': 2, 'High': 3})
data['Teacher_Quality'] = data['Teacher_Quality'].map({'Low': 1, 'Medium': 2, 'High': 3})
# Convert "Yes" to 1 and "No" to 0
data['Learning_Disabilities'] = data['Learning_Disabilities'].map({'Yes': 1, 'No': 0})
data['Internet_Access'] = data['Internet_Access'].map({'Yes': 1, 'No': 0})
# Convert Parental Education
data['Parental_Education_Level'] = data['Parental_Education_Level'].map({
    'High School': 1,
    'College': 2,
    'Postgraduate': 3
})
# Convert School Type
data['School_Type'] = data['School_Type'].map({'Private': 1, 'Public': 0})
# Convert Distance From Home
data['Distance_from_Home'] = data['Distance_from_Home'].map({
    'Near': 1,
    'Moderate': 2,
    'Far': 3
})
data.sample(10)

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Exam_Score
1569,17,69,2,2,7,70,1,1,0,1,3,0,1,2,2,61
3275,28,74,2,2,9,96,1,1,0,2,2,1,0,1,1,68
4307,24,93,2,1,7,77,2,1,0,1,2,1,0,1,1,68
707,31,63,1,3,7,74,1,1,3,2,2,0,0,2,2,66
5717,18,70,3,1,7,60,2,1,1,2,3,0,0,2,1,65
4280,33,63,3,3,9,59,2,1,1,2,2,0,0,1,1,69
6431,4,60,2,2,7,55,2,1,2,1,2,1,0,3,1,58
6158,21,83,3,3,7,64,2,1,2,2,1,1,0,1,1,68
2670,16,90,3,1,6,90,2,1,2,3,2,0,0,1,1,69
2117,14,81,1,2,8,71,1,1,1,1,3,0,1,3,1,63
